# Web scrape data

In [ ]:
# Webdriver - open the page and click on "Accept all"
from selenium import webdriver
from selenium.webdriver.common.by import By
import time 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import html.parser
import pandas as pd

In [2]:
def get_review_rate_time(result_set):
    rev_dict = {'Review Rate': [],
                'Review Time': []
                }
    for result in result_set:
        # find the review and time except when it is None type 
        try: 
            review_rate = result.find('span', class_='kvMYJc')['aria-label']
            rev_dict['Review Rate'].append(review_rate)

            review_time = result.find('span', class_='rsqaWe').text
            rev_dict['Review Time'].append(review_time)
        except TypeError or AttributeError:
            pass

    return(pd.DataFrame(rev_dict))

In [3]:
def get_review_text(result_set):
    rev_dict = {'Review Text': []
                }
    for result in result_set: 
        # get the review and only get the odd line (even lines are all None)
        review_text = result.find('span', class_='wiI7pd')
        if review_text is not None:
            review_text_content = review_text.text
            rev_dict['Review Text'].append(review_text_content)
        else: 
            rev_dict['Review Text'].append(None)
    return(pd.DataFrame(rev_dict).iloc[lambda x: x.index % 2==1])

In [ ]:
url_list = {
    # 'inglewood_plainpalais': 'https://www.google.ch/maps/place/Inglewood+Plainpalais/@46.1941363,6.1356256,17z/data=!3m1!4b1!4m6!3m5!1s0x478c7ad5ea8b6c3d:0xcab0358f5a95c111!8m2!3d46.1941364!4d6.140239!16s%2Fg%2F12hkt3wm3?hl=en'#,
    #'inglewood_eaux_vives': 'https://www.google.ch/maps/place/Inglewood+Eaux-Vives/@46.2019106,6.1543685,17z/data=!3m1!4b1!4m6!3m5!1s0x478c6536f77925e7:0xed4637560bf9d338!8m2!3d46.2019106!4d6.1569434!16s%2Fg%2F12hksttms?hl=en',
    #'inglewood_nyon': 'https://www.google.ch/maps/place/Inglewood+Nyon/@46.3841267,6.2336704,17z/data=!3m1!4b1!4m6!3m5!1s0x478c434c98ccd95f:0x502556c0d7231f8e!8m2!3d46.3841267!4d6.2362453!16s%2Fg%2F11tdqd8g86?hl=en',
    # 'inglewood_saint_laurent': 'https://www.google.ch/maps/place/Inglewood+Saint-Laurent/@46.5224191,6.613002,15z/data=!3m2!4b1!5s0x478c2e31fb50c131:0x1f39e5f530132aba!4m6!3m5!1s0x478c2e31f16f0a8f:0xb9d7991feb029033!8m2!3d46.5224206!4d6.6314558!16s%2Fg%2F11bvt4srnl?hl=en',
    # 'holy_cow_plainpalais': 'https://www.google.ch/maps/place/Holy+Cow!+Gourmet+Burger+Co.+GEN%C3%88VE+PLAINPALAIS/@46.1967473,6.1244618,15z/data=!3m1!4b1!4m6!3m5!1s0x478c7ad4dcb1422d:0x99319b6addd1dd64!8m2!3d46.1967488!4d6.1429156!16s%2Fg%2F11x9gt295?hl=en',
    # 'holy_cow_cornavin': 'https://www.google.ch/maps/place/Holy+Cow/@46.2103302,6.1416593,17z/data=!3m1!4b1!4m6!3m5!1s0x478c6527119302c7:0xb77114f95fafe45b!8m2!3d46.2103302!4d6.1442342!16s%2Fg%2F11b6_frwc3?hl=en',
    # 'funky_monkey_room': 'https://www.google.ch/maps/place/Funky+Monkey+Room/@46.2033229,6.1332716,17z/data=!3m1!4b1!4m6!3m5!1s0x478c653427014cdd:0xfee2adcb821b70e4!8m2!3d46.2033229!4d6.1358465!16s%2Fg%2F1wk7prys?hl=en',
    # 'hamburger_foundation_plainpalais': 'https://www.google.ch/maps/place/The+Hamburger+Foundation+-+Plainpalais/@46.1964521,6.1413115,17z/data=!3m1!4b1!4m6!3m5!1s0x478c7ad4c068e6c1:0xa55046e000f9e64d!8m2!3d46.1964521!4d6.1438864!16s%2Fg%2F11c2kx5jb4?hl=en',
    # 'hamburger_foundation_paquis': 'https://www.google.ch/maps/place/The+Hamburger+Foundation+-+P%C3%A2quis/@46.2114407,6.1482427,17z/data=!3m1!4b1!4m6!3m5!1s0x478c65247eeb1a1f:0x4a93b6b7e35200a2!8m2!3d46.2114407!4d6.1508176!16s%2Fg%2F1q63ckph5?hl=en',
    # 'hamburger_foundation_eaux_vives': 'https://www.google.ch/maps/place/The+Hamburger+Foundation+-+Eaux-Vives/@46.2039979,6.1553321,17z/data=!3m1!4b1!4m6!3m5!1s0x478c65d7af26ae0f:0x2cf166cdb2fdd639!8m2!3d46.2039979!4d6.157907!16s%2Fg%2F11h_xd81hh?hl=en',
    'black_tap': 'https://www.google.ch/maps/place/Black+Tap/@46.2014093,6.1502367,17z/data=!3m1!4b1!4m6!3m5!1s0x478c6532199536c3:0x1fa6ca815c2ba92f!8m2!3d46.2014093!4d6.1528116!16s%2Fg%2F11fzyxrlws?hl=en'
   }

for url in url_list:

    path = "C:\Program Files (x86)\chromedriver.exe"
    driver = webdriver.Chrome(path)
    
    # Get the url link and access it 
    driver.get(url_list[url])
    driver.implicitly_wait(10)

    # Click on agree all to open the map
    element = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span')
    element.click()

    # Get the total number of reviews 
    total_num_of_reviews = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span').text.split(' ')[0]
    total_num_of_reviews = total_num_of_reviews.replace(',','').replace('(','').replace(')','')
    total_num_of_reviews = int(total_num_of_reviews)
    print(total_num_of_reviews)

    # Google can load maximum 1139 reviews and then stops loading. 
    # We can divide the download into two tranches: 1) highest to lowest (1139 reviews); 2) lowest to hightest (the rest, maybe with some overlap)
    # rest_number_of_reviews = total_num_of_reviews - 1129 
    
    # Click on 'More reviews'
    more_reviews = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]')
    more_reviews.click()

    # Click on "sort" then "highest rating"
    sort = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/div[2]/button/span/span').click()
    highest = driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[3]').click()

    # Find the scrollable element
    scrollable_div = driver.find_element(By.XPATH, "//*[@id=\"QA0Szd\"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]") 

    # Scroll to load all reviews 
    for i in range(0, 113):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        if i%10 == 0: print(i)
        time.sleep(5)

    # Click on 'more' for each review
    items = driver.find_elements(By.CSS_SELECTOR, '.w8nwRe.kyuRq')

    for item in items:
        item.click()
        time.sleep(3)
    
    # Parsing HTML
    response = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = response.find_all('div', class_='GHT2ce')

    # Get the reviews (from highest to lowest)
    df_rate_time = get_review_rate_time(reviews)
    df_text = get_review_text(reviews)
    df_text.reset_index(drop=True, inplace=True)
    frames = [df_rate_time, df_text]
    df = pd.concat(frames, axis=1)

    # Save to csv 
    df.to_csv(f'{url}_highest.csv', encoding='utf-8-sig')

    # Click on "sort" then "lowest rating"
    sort = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/div[2]/button/span/span').click()
    highest = driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[4]').click()

    # Find the scrollable element
    scrollable_div = driver.find_element(By.XPATH, "//*[@id=\"QA0Szd\"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]") 

    # Calculate how many times to scroll down (each scroll = 10 reviews)
    # Scroll_num = round(rest_number_of_reviews/10) + 10 

    # Scroll to load all reviews 
    for i in range(0, 113):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        if i%10 == 0: print(i)
        time.sleep(5)

    # Parsing HTML
    response = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = response.find_all('div', class_='GHT2ce')

    df_rate_time = get_review_rate_time(reviews)
    df_text = get_review_text(reviews)
    df_text.reset_index(drop=True, inplace=True)
    frames = [df_rate_time, df_text]
    df = pd.concat(frames, axis=1)
    df.to_csv(f'{url}_lowest.csv', encoding='utf-8-sig')

    driver.quit()

C:\Users\Wenxia\AppData\Local\Temp\ipykernel_16832\588240425.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


2118
0
10
20
30
40
50
60
70
80
90
100
110
0
10
20
30
40
50
60
70
80
90
100
110


# Merge data

In [ ]:
import pandas as pd
import os
import glob
import re
import nltk
import matplotlib.pyplot as plt 
# nltk.download('wordnet')

import time 

import gensim 
from nltk.stem import WordNetLemmatizer

import numpy as np
import pickle

### Import data, add location, and combine them

In [ ]:
# get current working directory
current_path = os.getcwd()

# get the list of csv files 
csv_files = glob.glob(os.path.join(current_path, 'data', '*.csv'))

maindf = pd.DataFrame()
# loop over the list of csv files 
for file in csv_files:

    # read the csv file
    df = pd.read_csv(file, dtype={'Review Text': str})
    
    # get the file name and extract the location info 
    file_name = os.path.basename(file)
    removedWord = ['inglewood', 'highest.csv', 'lowest.csv']
    location = ' '.join(i for i in file_name.split('_') if i not in removedWord).title()
    
    # add a location column to the data frame 
    df['Location'] = location
    # concat all the df 
    maindf = pd.concat([maindf, df], axis=0)

In [ ]:
# drop the first column (old index from csv)
maindf.drop('Unnamed: 0', axis=1, inplace=True)

# drop duplicates 
maindf.drop_duplicates(subset = 'Review Text', inplace=True)

In [ ]:
# save the data
maindf.head()
maindf.to_csv('merged_raw_data.csv')